In [1]:
%load_ext pycodestyle_magic

In [18]:
#%%pycodestyle
# rate_increase.py

import spacy
import pandas as pd
from collections import Counter

# 使用する単語の品詞とストップワードの指定
include_pos = ('NOUN', 'PROPN')
stopwords = ('お', 'こと', 'ため', 'なり', 'もの', 'よう', 'ほう')


# ユニバーサル・スタジオ・ジャパンの口コミのファイルの読み込み
df = pd.read_csv('text/usj.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

# 旅行の時期が欠損している口コミを削除する
df = df.dropna(subset=['旅行の時期'])

# 旅行の時期を年度に直し，「年度」という項目を追加
df['年度'] = df['旅行の時期'].apply(lambda x: x.year if x.month >= 4 else x.year-1)

# 2011年度～2013年度までと、2014年度から2018年度までの口コミに分ける
df2013 = df[(df['年度'] >= 2011) & (df['年度'] <= 2013)]
df2014 = df[(df['年度'] >= 2014) & (df['年度'] <= 2018)]

nlp = spacy.load("ja_ginza")


# 口コミに出現する頻度をカウントする関数
def count_words(df):
    # 出現する単語をリストに追加
    words = []
    for doc in nlp.pipe(df["口コミ"]):
        words.extend([token.lemma_ for token in doc
                      if token.pos_ in include_pos and
                      token.lemma_ not in stopwords])

    # 単語の頻度をカウント
    counter = Counter(words)
    
    return counter

In [19]:
# 単語の頻度をカウント
counter2013 = count_words(df2013)   # 2013年度までの単語の頻度をカウント
counter2014 = count_words(df2014)   # 2014年度までの単語の頻度をカウント

In [20]:
# 単語の増加率を算出
word2rate = Counter()                    # 単語と増加率を保存する辞書
for word, count in counter2014.items():
    if len(word) > 1 and count >= 100:
        if word in counter2013:
            rate = count / counter2013[word]
            if rate >= 3.0:
                word2rate[word] = round(rate, 1)
        else:
            # 2013年度までの口コミに単語が出現しない場合は増加率を1000としておく
            word2rate[word] = 1000.0

# 増加率の順番に表示
print('    rate word')
for word, rate in word2rate.most_common():
    print(f'{rate:>8} {word}')

    rate word
  1000.0 ホグワーツ
  1000.0 ミニオン
  1000.0 バター
  1000.0 フライングダイナソー
   109.0 ハリポタ
    65.5 ビール
    53.7 ハリー
    53.6 ポッター
    20.4 整理券
    16.5 エクスプレスパス
    10.3 行列
    10.2 エリア
     9.2 入場
     7.3 なし
     6.5 エクスプレス
     6.3 本当
     6.0 世界
     5.4 お土産
     5.4 事前
     5.3 お金
     5.2 雰囲気
     5.0 購入
     4.9 今回
     4.9 人気
     4.9 夕方
     4.8 レストラン
     4.7 値段
     4.5 以上
     4.4 ディズニー
     4.4 乗り物
     4.4 混雑
     4.3 さん
     4.3 時間
     4.1 映画
     4.1 チケット
     4.1 待ち
     4.1 有料
     4.0 パーク
     4.0 食事
     4.0 感じ
     4.0 開園
     4.0 時期
     4.0 ホテル
     3.9 USJ
     3.9 スタッフ
     3.8 ライド
     3.8 家族
     3.8 ハロウィン
     3.7 テーマ
     3.7 アトラクション
     3.7 パス
     3.7 クリスマス
     3.7 以外
     3.7 ゾンビ
     3.5 ジュラシック
     3.4 場所
     3.4 大阪
     3.3 一度
     3.3 イベント
     3.2 ファストパス
     3.2 平日


In [21]:
#%%pycodestyle
# count_word_usj.py

# 年度ごとに単語の頻度をカウント
year2counter = {}
for y in range(2011, 2019):
    year2counter[y] = count_words(df[df['年度'] == y])

In [22]:
# カウントするキーワードを登録した辞書を用意
keywords = {'ハリーポッター': ['ハリーポッター', 'ハリポタ', 'ハリー'],
            'ミニオン': ['ミニオン', 'ミニオンズ', 'ミニオンパーク'],
            'フライングダイナソー': ['フライングダイナソー'],
            'ハロウィン': ['ハロウィン', 'ハロウィーン', 'ハローウィン', 'ハローイン', 'ハロイン'],
            'クリスマス': ['クリスマス']}

In [23]:
# 年度ごとに単語の頻度を表示
records = []
for keyword, synonyms in keywords.items():
    record = {'キーワード': keyword}
    for y in range(2011, 2019):
        count = sum([year2counter[y][s] for s in synonyms])
        record[y] = count
    records.append(record)

df_res = pd.DataFrame(records)  # 単語の頻度を保存するデータフレーム
df_res = df_res.set_index('キーワード')
display(df_res)

,2011,2012,2013,2014,2015,2016,2017,2018
キーワード,,,,,,,,
ハリーポッター,1,4,22,554,519,303,158,85
ミニオン,0,0,0,0,5,22,90,39
フライングダイナソー,0,0,0,0,4,76,23,20
ハロウィン,6,14,22,26,52,40,22,14
クリスマス,7,11,15,26,38,34,17,8


*.pyファイルの表示用

In [24]:
pd.set_option('display.unicode.east_asian_width', True)
print(df_res)

                      2011  2012  2013  2014  2015  2016  2017  2018
キーワード                                                          
ハリーポッター           1     4    22   554   519   303   158    85
ミニオン                 0     0     0     0     5    22    90    39
フライングダイナソー     0     0     0     0     4    76    23    20
ハロウィン               6    14    22    26    52    40    22    14
クリスマス               7    11    15    26    38    34    17     8
